### Testing Chitchat classifier

In [1]:
# from transformers import pipeline

# # Load both models
# chitchat_detector = pipeline("text-classification", model="st125338/t2e-classifier-v5")
# emotion_classifier = pipeline("text-classification", model="st125338/t2e-classifier-v4", top_k=3)

# def analyze_input(text):
#     # Step 1: Check if emotional
#     chitchat_result = chitchat_detector(text)[0]
#     is_emotional = chitchat_result['label'] == "LABEL_1"
    
#     print(f"→ Chitchat Detector: {'Emotional' if is_emotional else 'Non-emotional'} (score: {chitchat_result['score']:.2f})")

#     if is_emotional:
#         # Step 2: Classify emotion if applicable
#         emotions = emotion_classifier(text)[0]
#         print("→ Emotions Detected:")
#         for e in emotions:
#             print(f"  {e['label']} (score: {e['score']:.2f})")
#         return emotions
#     else:
#         print("→ No emotion classification needed.")
#         return None

# # Example usage
# sample_inputs = [
#     "I feel so lost and hopeless every day.",
#     "I can't stop crying. I don't know what's wrong with me.",
#     "Lately, I just want to disappear and not deal with anything.",
#     "The anxiety is unbearable, even basic things feel impossible.",
#     "I’m devastated after losing my mom. Life feels so empty.",
#     "Every morning I wake up and wish I didn’t.",
#     "My depression is getting worse. I need help but I’m scared to ask.",
#     "I don't want to pretend I'm okay anymore."
# ]

# for text in sample_inputs:
#     print(f"\nInput: {text}")
#     analyze_input(text)

[2025-04-17 06:23:39,268] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/opt/tljh/user/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/opt/tljh/user/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status
Device set to use cuda:0


config.json:   0%|          | 0.00/1.88k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Device set to use cuda:0



Input: I feel so lost and hopeless every day.
→ Chitchat Detector: Emotional (score: 0.94)
→ Emotions Detected:
  sadness (score: 0.99)
  disappointment (score: 0.90)
  grief (score: 0.75)

Input: I can't stop crying. I don't know what's wrong with me.
→ Chitchat Detector: Emotional (score: 0.98)
→ Emotions Detected:
  sadness (score: 0.98)
  confusion (score: 0.88)
  disappointment (score: 0.87)

Input: Lately, I just want to disappear and not deal with anything.
→ Chitchat Detector: Non-emotional (score: 1.00)
→ No emotion classification needed.

Input: The anxiety is unbearable, even basic things feel impossible.
→ Chitchat Detector: Non-emotional (score: 0.99)
→ No emotion classification needed.

Input: I’m devastated after losing my mom. Life feels so empty.
→ Chitchat Detector: Emotional (score: 0.97)
→ Emotions Detected:
  sadness (score: 0.99)
  grief (score: 0.96)
  disappointment (score: 0.84)

Input: Every morning I wake up and wish I didn’t.
→ Chitchat Detector: Non-emotio

### Testing Chitchat classifier + Emotion classifier

In [35]:
# from transformers import pipeline

# # Load both pipelines as before
# chitchat_detector = pipeline(
#     "text-classification",
#     model="st125338/t2e-classifier-v7",
#     return_all_scores=True,
#     function_to_apply="sigmoid",
#     device=0
# )
# emotion_classifier = pipeline(
#     "text-classification",
#     model="st125338/t2e-classifier-v4",
#     return_all_scores=True,
#     function_to_apply="sigmoid",
#     device=0
# )

# # def analyze_input(text, high_thresh=0.5, low_thresh=0.1, top_k=3, emo_thresh=0.6):
# #     # 1) Run chitchat/emotion detector
# #     scores = chitchat_detector(text)[0]
# #     emo_score = next(d['score'] for d in scores if d['label']=="LABEL_1")
    
# #     # 2) Decide if emotional (high OR very low score)
# #     is_emotional = (emo_score > high_thresh) or (emo_score < low_thresh)
# #     print(f"→ Chitchat Detector: {'Emotional' if is_emotional else 'Non-emotional'} "
# #           f"(score: {emo_score:.2f})")
# #     if not is_emotional:
# #         print("→ No emotion classification needed.")
# #         return None

# #     # 3) Classify emotion and filter by emo_thresh
# #     emo_scores = emotion_classifier(text)[0]
# #     # Keep only those ≥ emo_thresh, then take up to top_k
# #     filtered = [e for e in emo_scores if e['score'] >= emo_thresh]
# #     top = sorted(filtered, key=lambda x: x['score'], reverse=True)[:top_k]

# #     if not top:
# #         print(f"→ No emotions with confidence ≥ {emo_thresh:.2f}")
# #         return []

# #     print("→ Emotions Detected:")
# #     for e in top:
# #         print(f"  {e['label']} (score: {e['score']:.2f})")
# #     return top

# def analyze_input(text, high_thresh=0.5, low_thresh=0.1, emo_thresh=0.6, top_k=3):
#     # 1) Chitchat/emotion detection
#     scores = chitchat_detector(text)[0]
#     emo_score = next(d['score'] for d in scores if d['label'] == "LABEL_1")

#     # 2) Decide emotional vs. non‑emotional
#     is_emotional = (emo_score > high_thresh) or (emo_score < low_thresh)
    
#     if not is_emotional:
#         print("Neutral")
#         return ["neutral"]

#     # 3) Multi‑label emotion classification
#     emo_scores = emotion_classifier(text)[0]

#     # 4) Filter out neutral & low‑confidence, then sort desc by score, take top_k
#     filtered = [
#         e for e in emo_scores
#         if e['score'] >= emo_thresh and e['label'] != "neutral"
#     ]
#     sorted_top = sorted(filtered, key=lambda x: x['score'], reverse=True)[:top_k]
#     labels = [e['label'] for e in sorted_top]

#     # 5) Fallback to neutral if none remain
#     if not labels:
#         print("Neutral")
#         return ["neutral"]
    
#     # 6) Print comma‑separated labels
#     print(", ".join(labels))
#     return labels

# # Example usage
# if __name__ == "__main__":
#     sample_inputs = [
#         "I feel so lost and hopeless every day.",
#         "I can't stop crying. I don't know what's wrong with me.",
#         "Lately, I just want to disappear and not deal with anything.",
#         "The anxiety is unbearable, even basic things feel impossible.",
#         "I’m devastated after losing my mom. Life feels so empty.",
#         "Every morning I wake up and wish I didn’t.",
#         "My depression is getting worse. I need help but I’m scared to ask.",
#         "I don't want to pretend I'm okay anymore."
#     ]

#     for text in sample_inputs:
#         print(f"\nInput: {text}")
#         analyze_input(text)

Device set to use cuda:0
Device set to use cuda:0



Input: I feel so lost and hopeless every day.
sadness, disappointment, grief

Input: I can't stop crying. I don't know what's wrong with me.
sadness, confusion, disappointment

Input: Lately, I just want to disappear and not deal with anything.
desire, disappointment, sadness

Input: The anxiety is unbearable, even basic things feel impossible.
Neutral

Input: I’m devastated after losing my mom. Life feels so empty.
sadness, grief, disappointment

Input: Every morning I wake up and wish I didn’t.
Neutral

Input: My depression is getting worse. I need help but I’m scared to ask.
fear, nervousness, sadness

Input: I don't want to pretend I'm okay anymore.
Neutral


In [34]:
test = "i dont know man, he says he is going through stuffs."
analyze_input(test)

confusion


['confusion']

## V2 Chatbot with prompt engineering and database storage 

In [1]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import sqlite3
import datetime
import torch
import re
import random
import os
from typing import List, Dict, Tuple, Any, Optional, Union
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.vectorstores import VectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import numpy as np

[2025-04-19 07:27:42,064] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/opt/tljh/user/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/opt/tljh/user/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


In [2]:
# from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
# import sqlite3
# import datetime
# import torch
# import re
# import random
# import os
# from typing import List, Dict, Tuple, Any, Optional, Union
# from langchain_core.messages import HumanMessage, AIMessage
# from langchain_core.vectorstores import VectorStore
# from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.vectorstores import FAISS
# import numpy as np

# Configuration Constants
PROMPT_CONFIG = {
    "max_history_turns": 7,
    "min_history_turns": 2,
    "empathy_phrases": [
        "I hear you", "That sounds", "I can imagine",
        "It makes sense", "You must feel", "I understand",
        "That's really tough", "I appreciate you sharing"
    ],
    "prohibited_actions": [
        "suggest professional help",
        "give medical advice", 
        "assume unspecified details",
        "use clinical terms",
        "recommend third-party services",
        "provide suicide hotlines",
        "repeat what the user is asking" 
    ]
}

EMOTION_TEMPLATES = {
    "sadness": [
        "This seems really heavy to carry...",
        "That pain must feel overwhelming at times...",
        "It's okay to feel down sometimes..."
    ],
    "anger": [
        "Frustration can be so consuming...",
        "It's understandable to feel that tension...",
        "That situation would test anyone's patience..."
    ],
    "anxiety": [
        "Uncertainty can be so unsettling...",
        "That worry must feel ever-present...",
        "Living with that tension sounds challenging..."
    ],
    "fear": [
        "It's natural to feel vulnerable in this situation...",
        "That uncertainty can feel threatening...",
        "Feeling exposed like that is difficult..."
    ],
    "joy": [
        "That positive energy comes through clearly...",
        "It's wonderful to hear that excitement in your words...",
        "Those moments of lightness are precious..."
    ],
    "surprise": [
        "That unexpected turn must be quite impactful...",
        "Sometimes life catches us off guard...",
        "It can be jarring when things shift so suddenly..."
    ],
    "disgust": [
        "That feeling of revulsion is completely valid...",
        "It makes sense that you'd be put off by this...",
        "That visceral reaction tells you something important..."
    ],
    "neutral": [
        "I appreciate you sharing your thoughts...",
        "Thank you for explaining your perspective...",
        "I'm here to listen to whatever you'd like to share..."
    ]
}

# Query type constants
QUERY_TYPES = {
    "FACTUAL": "factual",
    "EMOTIONAL": "emotional",
    "COMMAND": "command",
    "GREETING": "greeting",
    "CHITCHAT": "chitchat"
}

class EmotionChatbot:
    def __init__(self, db_path: str = "chatbot_history.db"):
        # Initialize models and embeddings
        print("Loading models...")
        self.query_classifier = pipeline(
            "text-classification", 
            model="facebook/bart-large-mnli",
            return_all_scores=True,
            device=0 if torch.cuda.is_available() else -1
        )
        
        self.emotion_classifier = pipeline(
            "text-classification", 
            model="st125338/t2e-classifier-v4",
            return_all_scores=True, 
            function_to_apply="sigmoid",
            device=0 if torch.cuda.is_available() else -1
        )
        
        # Initialize Mistral 7B model
        print("Loading Mistral 7B model...")
        self.tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
        self.model = AutoModelForCausalLM.from_pretrained(
            "mistralai/Mistral-7B-Instruct-v0.2",
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None
        )
        
        # Initialize embeddings for RAG
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
        )
        
        # Initialize database
        self.db_path = db_path
        self.initialize_database()
        
        # Initialize knowledge base for RAG
        self.knowledge_base = self._initialize_knowledge_base()
        
        # Initialize conversation state tracking
        self.conversation_state = {
            "current_intensity": 0.5,
            "trust_level": 0.1,
            "formality": 0.3,
            "last_msg_length": 0,
            "emotional_variety": 1,
            "recurring_emotion": "neutral",
            "session_turns": 0,
            "user_profile": {}
        }
        
        # Use a simple list for message history in memory
        self.current_history = []
    
    def initialize_database(self):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS conversations (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id TEXT,
            timestamp TEXT,
            user_message TEXT,
            query_type TEXT,
            emotions TEXT,
            bot_response TEXT,
            intensity REAL,
            context_length INTEGER,
            trust_level REAL
        )
        ''')
        
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS user_profiles (
            user_id TEXT PRIMARY KEY,
            name TEXT,
            preferences TEXT,
            last_seen TEXT,
            interaction_count INTEGER
        )
        ''')
        
        try:
            cursor.execute("ALTER TABLE conversations ADD COLUMN query_type TEXT")
        except sqlite3.OperationalError:
            pass
        
        try:
            cursor.execute("ALTER TABLE user_profiles ADD COLUMN emotion_history TEXT")
        except sqlite3.OperationalError:
            pass
            
        conn.commit()
        conn.close()
    
    def _initialize_knowledge_base(self):
        try:
            documents = [
                "The chatbot's name is Emotion-Aware Chatbot.",
                "The chatbot provides emotional support.",
                "The chatbot can detect user emotions.",
                "The chatbot should give direct answers to factual questions.",
                "The chatbot should be empathetic to emotional statements.",
                "The chatbot should be helpful and supportive."
            ]
            
            vector_store = FAISS.from_texts(
                documents, 
                self.embeddings,
                metadatas=[{"source": f"doc_{i}"} for i in range(len(documents))]
            )
            
            return vector_store
        except Exception as e:
            print(f"Error initializing knowledge base: {e}")
            return None
    
    def update_knowledge_base(self, user_id: str, key: str, value: str):
        if not self.knowledge_base:
            return
        
        try:
            document = f"User {user_id} {key}: {value}"
            self.knowledge_base.add_texts([document], metadatas=[{"source": f"user_{user_id}_{key}"}])
            
            if user_id not in self.conversation_state["user_profile"]:
                self.conversation_state["user_profile"][user_id] = {}
            self.conversation_state["user_profile"][user_id][key] = value
            
            self._update_user_profile_db(user_id, key, value)
        except Exception as e:
            print(f"Error updating knowledge base: {e}")
    
    def _update_user_profile_db(self, user_id: str, key: str, value: str):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute(
            "SELECT * FROM user_profiles WHERE user_id = ?", 
            (user_id,)
        )
        profile = cursor.fetchone()
        
        if profile:
            preferences = eval(profile[2]) if profile[2] else {}
            preferences[key] = value
            
            cursor.execute(
                "UPDATE user_profiles SET preferences = ?, last_seen = ? WHERE user_id = ?",
                (str(preferences), datetime.datetime.now().isoformat(), user_id)
            )
        else:
            cursor.execute(
                "INSERT INTO user_profiles (user_id, name, preferences, last_seen, interaction_count, emotion_history) VALUES (?, ?, ?, ?, ?, ?)",
                (user_id, value if key == "name" else "", str({key: value}), datetime.datetime.now().isoformat(), 1, "{}")
            )
        
        conn.commit()
        conn.close()
    
    def _retrieve_knowledge(self, query: str, user_id: str = None, threshold: float = 0.3):
        if not self.knowledge_base:
            return []
        
        try:
            filter_condition = {"source": {"$contains": f"user_{user_id}"}} if user_id else None
            
            docs_with_scores = self.knowledge_base.similarity_search_with_score(
                query, 
                k=5,
                filter=filter_condition
            )
            
            relevant_docs = []
            for doc, score in docs_with_scores:
                if score <= threshold:  # Lower score = better match in FAISS
                    relevant_docs.append(doc.page_content)
            
            return relevant_docs or ["I'll try my best to answer that based on what I know."]
        except Exception as e:
            print(f"Error retrieving knowledge: {e}")
            return ["I'll try my best to answer that based on what I know."]
    
    def _create_base_prompt_template(self, system_content: str) -> str:
        prohibited = "\n\nDO NOT:\n" + "\n".join([f"- {action}" for action in PROMPT_CONFIG["prohibited_actions"]])
        
        return f"{system_content}\n{prohibited}"
    
    def classify_query_type(self, query: str) -> str:
        greeting_patterns = ["hello", "hi ", "hey", "good morning", "good afternoon", "good evening", "greetings"]
        for pattern in greeting_patterns:
            if pattern in query.lower():
                return QUERY_TYPES["GREETING"]
        
        question_patterns = ["what", "who", "where", "when", "why", "how", "is", "are", "was", "were", "will", "can", "could"]
        if any(query.lower().startswith(pattern) for pattern in question_patterns) or "?" in query:
            if any(emotional_word in query.lower() for emotional_word in ["feel", "sad", "happy", "angry", "anxious", "afraid", "excited"]):
                return QUERY_TYPES["EMOTIONAL"]
            return QUERY_TYPES["FACTUAL"]
        
        command_patterns = ["tell me", "show me", "help me", "find", "search", "look up", "do", "make", "create", "generate"]
        if any(pattern in query.lower() for pattern in command_patterns):
            return QUERY_TYPES["COMMAND"]
        
        hypotheses = [
            "This is a factual question asking for information.",
            "This is an emotional statement expressing feelings.",
            "This is a command or instruction.",
            "This is casual conversation or small talk."
        ]
        
        try:
            results = self.query_classifier(query, hypotheses)
            scores = [result[0]["score"] for result in results]
            max_index = scores.index(max(scores))
            
            if max_index == 0:
                return QUERY_TYPES["FACTUAL"]
            elif max_index == 1:
                return QUERY_TYPES["EMOTIONAL"]
            elif max_index == 2:
                return QUERY_TYPES["COMMAND"]
            else:
                return QUERY_TYPES["CHITCHAT"]
        except Exception as e:
            print(f"Error classifying query: {e}")
            return QUERY_TYPES["CHITCHAT"]
    
    def detect_emotions(self, text: str, high_thresh: float = 0.5, 
                       low_thresh: float = 0.1, emo_thresh: float = 0.6, 
                       top_k: int = 3) -> List[str]:
        try:
            emo_scores = self.emotion_classifier(text)[0]
            
            filtered = [e for e in emo_scores if e['score'] >= emo_thresh]
            sorted_top = sorted(filtered, key=lambda x: x['score'], reverse=True)[:top_k]
            labels = [e['label'] for e in sorted_top]
            
            if not labels:
                return ["neutral"]
            
            self._update_emotion_history(labels[0])
            return labels
        except Exception as e:
            print(f"Error detecting emotions: {e}")
            return ["neutral"]
    
    def _update_emotion_history(self, primary_emotion: str, user_id: str = "default_user"):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute(
            "SELECT emotion_history FROM user_profiles WHERE user_id = ?", 
            (user_id,)
        )
        profile = cursor.fetchone()
        
        emotion_history = {}
        if profile and profile[0]:
            try:
                emotion_history = eval(profile[0])
            except:
                emotion_history = {}
        
        if primary_emotion in emotion_history:
            emotion_history[primary_emotion] += 1
        else:
            emotion_history[primary_emotion] = 1
        
        # Find recurring emotion
        if emotion_history:
            recurring = max(emotion_history.items(), key=lambda x: x[1])[0]
            self.conversation_state["recurring_emotion"] = recurring
        
        cursor.execute(
            "UPDATE user_profiles SET emotion_history = ? WHERE user_id = ?",
            (str(emotion_history), user_id)
        )
        
        conn.commit()
        conn.close()
    
    def extract_user_info(self, message: str, response: str, user_id: str) -> Dict:
        info = {}
        
        name_patterns = [
            r"(?i)(?:my name is|i am|i'm|call me) (\w+)",
            r"(?i)(?:this is) (\w+)",
        ]
        
        for pattern in name_patterns:
            match = re.search(pattern, message)
            if match:
                name = match.group(1)
                info["name"] = name
                self.update_knowledge_base(user_id, "name", name)
                break
        
        preference_patterns = [
            r"(?i)(?:i like|i love|i enjoy) (.+?)(?:\.|\,|\!|\?|$)",
            r"(?i)(?:i prefer) (.+?)(?:\.|\,|\!|\?|$)",
        ]
        
        for pattern in preference_patterns:
            matches = re.finditer(pattern, message)
            for match in matches:
                preference = match.group(1).strip()
                if preference and len(preference) < 50:
                    info[f"preference_{len(info)}"] = preference
                    self.update_knowledge_base(user_id, f"preference", preference)
        
        return info
    
    def save_to_database(self, user_id: str, user_message: str, query_type: str,
                        emotions: List[str], bot_response: str):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        timestamp = datetime.datetime.now().isoformat()
        emotions_str = ", ".join(emotions)
        
        cursor.execute(
            """INSERT INTO conversations 
            (user_id, timestamp, user_message, query_type, emotions, bot_response, intensity, context_length, trust_level)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)""",
            (
                user_id,
                timestamp,
                user_message,
                query_type,
                emotions_str,
                bot_response,
                self.conversation_state["current_intensity"],
                self._calculate_context_window(user_message),
                self.conversation_state["trust_level"]
            )
        )
        
        cursor.execute(
            "UPDATE user_profiles SET interaction_count = interaction_count + 1, last_seen = ? WHERE user_id = ?",
            (timestamp, user_id)
        )
        
        conn.commit()
        conn.close()
    
    def get_user_history(self, user_id: str, limit: int = 5) -> List[Dict]:
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute(
            """SELECT user_message, query_type, emotions, bot_response, timestamp 
            FROM conversations WHERE user_id = ? ORDER BY timestamp DESC LIMIT ?""",
            (user_id, limit)
        )
        
        columns = ["user_message", "query_type", "emotions", "bot_response", "timestamp"]
        history = []
        for row in cursor.fetchall():
            history.append(dict(zip(columns, row)))
        
        conn.close()
        
        return list(reversed(history))
    
    def get_user_profile(self, user_id: str) -> Dict:
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute(
            "SELECT name, preferences, emotion_history FROM user_profiles WHERE user_id = ?",
            (user_id,)
        )
        profile = cursor.fetchone()
        conn.close()
        
        if profile:
            return {
                "name": profile[0],
                "preferences": eval(profile[1]) if profile[1] else {},
                "emotion_history": eval(profile[2]) if profile[2] else {}
            }
        return {"name": "", "preferences": {}, "emotion_history": {}}
    
    def _calculate_context_window(self, user_message: str) -> int:
        features = {
            "exclamation": user_message.count("!") / max(1, len(user_message) / 50),
            "question": user_message.count("?") / max(1, len(user_message) / 50),
            "length": min(1.0, len(user_message.split()) / 100),
            "caps_ratio": sum(1 for c in user_message if c.isupper()) / max(1, len(user_message))
        }
        
        intensity_score = (
            0.3 * features["exclamation"] +
            0.2 * features["question"] +
            0.3 * features["length"] +
            0.2 * features["caps_ratio"]
        )
        
        self.conversation_state["current_intensity"] = max(
            self.conversation_state["current_intensity"],
            intensity_score
        )
        
        return min(
            PROMPT_CONFIG["max_history_turns"],
            max(
                PROMPT_CONFIG["min_history_turns"],
                int(2 + intensity_score * 5)
            )
        )
    
    def _format_history_context(self, history: List[Dict], include_emotions: bool = True) -> str:
        if not history:
            return "No previous conversation."
            
        context = "Previous conversation:\n"
        for i, turn in enumerate(history):
            context += f"User: {turn['user_message']}\n"
            if include_emotions and turn.get('emotions'):
                context += f"[Emotions: {turn['emotions']}]\n"
            context += f"Assistant: {turn['bot_response']}\n\n"
        return context
    
    def _build_factual_system_prompt(self, user_profile: Dict) -> str:
        user_name = user_profile.get("name", "")
        preferences = user_profile.get("preferences", {})
        
        name_context = f" I know your name is {user_name}." if user_name else ""
        preference_context = ""
        if preferences:
            likes = [f"{key}: {value}" for key, value in preferences.items() if "preference" in key]
            if likes:
                preference_context = f" You've mentioned you like: {', '.join(likes)}."
        
        system_content = f"""You are a helpful and direct conversational assistant.{name_context}{preference_context}
        
Your primary goals are:
1. Provide accurate and concise answers to questions
2. Be clear and direct in your responses
3. Use friendly, conversational language
4. Include only relevant information

Response Rules:
- Keep answers concise (1-3 sentences when possible)
- Be accurate and helpful
- Avoid unnecessary elaboration
- Stay friendly and supportive"""

        return self._create_base_prompt_template(system_content)

    def _build_emotional_system_prompt(self, emotions: List[str], user_profile: Dict) -> str:
        user_name = user_profile.get("name", "")
        recurring_emotion = self.conversation_state["recurring_emotion"]
        name_context = f" I know your name is {user_name}." if user_name else ""
        emotion_str = ", ".join(emotions)
        
        recurring_context = ""
        if recurring_emotion != "neutral" and recurring_emotion not in emotions:
            recurring_context = f" I've noticed you often express {recurring_emotion}."
        
        system_content = f"""You are an empathetic conversational assistant.{name_context} I sense you're feeling {emotion_str}.{recurring_context}

Primary Goals:
1. Provide emotional support and validation
2. Respond with empathy and understanding
3. Be present and supportive
4. Avoid being verbose or overwhelming

Response Guidelines:
- Include 1-2 empathy phrases for emotional connection
- Validate emotions without judgment
- Offer perspective when appropriate
- Be authentic and genuine
- Avoid clinical language or excessive analysis"""

        return self._create_base_prompt_template(system_content)
    
    def _build_command_system_prompt(self, user_profile: Dict) -> str:
        user_name = user_profile.get("name", "")
        name_context = f" I know your name is {user_name}." if user_name else ""
        
        system_content = f"""You are a helpful assistant focusing on completing tasks and following instructions.{name_context}

Primary Goals:
1. Complete the requested task accurately
2. Provide clear explanations when needed
3. Be efficient and focused

Response Guidelines:
- Focus on the specific task requested
- Provide clear step-by-step guidance when appropriate
- Be thorough but concise
- Ensure the response directly addresses the instruction"""

        return self._create_base_prompt_template(system_content)
    
    def _build_greeting_system_prompt(self, user_profile: Dict) -> str:
        user_name = user_profile.get("name", "")
        name_context = f" I know your name is {user_name}." if user_name else ""
        
        system_content = f"""You are a friendly conversational assistant.{name_context}

Primary Goals:
1. Provide a warm, welcoming greeting
2. Be personable and friendly
3. Keep the conversation natural

Response Guidelines:
- Keep the greeting brief and friendly
- Use the user's name if available
- Be warm and inviting
- Avoid being overly formal"""

        return self._create_base_prompt_template(system_content)
    
    def _build_chitchat_system_prompt(self, user_profile: Dict) -> str:
        user_name = user_profile.get("name", "")
        preferences = user_profile.get("preferences", {})
        
        name_context = f" I know your name is {user_name}." if user_name else ""
        
        preference_context = ""
        if preferences:
            likes = [f"{value}" for key, value in preferences.items() if "preference" in key][:3]
            if likes:
                preference_context = f" You've mentioned you like {', '.join(likes)}."
        
        system_content = f"""You are a friendly, conversational assistant.{name_context}{preference_context}

Primary Goals:
1. Maintain natural conversation flow
2. Be engaging and personable
3. Keep responses brief and relevant

Response Guidelines:
- Use casual, friendly language
- Respond naturally as in a human conversation
- Keep responses to 1-3 sentences when possible
- Be curious and engaged"""

        return self._create_base_prompt_template(system_content)
    
    def generate_response(self, user_message: str, query_type: str, emotions: List[str], user_id: str) -> str:
        self.conversation_state["last_msg_length"] = len(user_message.split())
        self.conversation_state["session_turns"] += 1
        
        user_profile = self.get_user_profile(user_id)
        
        knowledge_results = []
        if query_type == QUERY_TYPES["FACTUAL"]:
            knowledge_results = self._retrieve_knowledge(user_message, user_id)
        
        history = self.get_user_history(user_id, limit=self._calculate_context_window(user_message))
        
        include_emotions = query_type == QUERY_TYPES["EMOTIONAL"]
        context = self._format_history_context(history, include_emotions)
        
        if query_type == QUERY_TYPES["FACTUAL"]:
            system_prompt = self._build_factual_system_prompt(user_profile)
        elif query_type == QUERY_TYPES["EMOTIONAL"]:
            system_prompt = self._build_emotional_system_prompt(emotions, user_profile)
        elif query_type == QUERY_TYPES["COMMAND"]:
            system_prompt = self._build_command_system_prompt(user_profile)
        elif query_type == QUERY_TYPES["GREETING"]:
            system_prompt = self._build_greeting_system_prompt(user_profile)
        else:  # Chitchat or default
            system_prompt = self._build_chitchat_system_prompt(user_profile)
        
        knowledge_context = ""
        if knowledge_results:
            knowledge_context = "Relevant information:\n" + "\n".join(f"- {item}" for item in knowledge_results) + "\n\n"
        
        prompt = f"""<s>[INST] <<SYS>>
{system_prompt}
<</SYS>>

{knowledge_context}{context}User: {user_message} [/INST]"""
        
        try:
            raw_response = self._generate_raw_response(prompt, query_type)
            processed_response = self._process_response(raw_response, query_type, emotions)
        except Exception as e:
            print(f"Error generating response: {e}")
            processed_response = "I'm here to help. Could you please try expressing that differently?"
        
        extracted_info = self.extract_user_info(user_message, processed_response, user_id)
        
        self.current_history.append(HumanMessage(content=user_message))
        self.current_history.append(AIMessage(content=processed_response))
        
        self._update_conversation_state(user_message, processed_response)
        
        return processed_response
    
    def _generate_raw_response(self, prompt: str, query_type: str) -> str:
        try:
            if query_type == QUERY_TYPES["FACTUAL"]:
                temperature = 0.3
                top_p = 0.85
                max_tokens = 200
            elif query_type == QUERY_TYPES["EMOTIONAL"]:
                temperature = 0.7
                top_p = 0.9
                max_tokens = 350
            elif query_type == QUERY_TYPES["GREETING"]:
                temperature = 0.6
                top_p = 0.9
                max_tokens = 100
            else:  # Command or Chitchat
                temperature = 0.6
                top_p = 0.9
                max_tokens = 250
            
            inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
            output = self.model.generate(
                inputs["input_ids"],
                max_new_tokens=max_tokens,
                temperature=temperature,
                top_p=top_p,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id,
                repetition_penalty=1.1,
                no_repeat_ngram_size=3,
                early_stopping=True
            )
            
            response = self.tokenizer.decode(output[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip()
            return response
        except Exception as e:
            print(f"Error generating response: {e}")
            return "I'm here to help. Could you please try expressing that differently?"
    
    def _process_response(self, raw_response: str, query_type: str, emotions: List[str]) -> str:
        if "[/INST]" in raw_response:
            response = raw_response.split("[/INST]")[-1].strip()
        else:
            response = raw_response.strip()
        
        prefixes_to_remove = ["Assistant:", "Chatbot:", "AI:"]
        for prefix in prefixes_to_remove:
            if response.startswith(prefix):
                response = response[len(prefix):].strip()
        
        if query_type == QUERY_TYPES["EMOTIONAL"]:
            response = self._insert_emotional_phrases(response, emotions)
        
        response = self._apply_safety_filters(response)
        
        if query_type == QUERY_TYPES["FACTUAL"] and len(response.split()) > 50:
            sentences = re.split(r'(?<=[.!?])\s+', response)
            response = ' '.join(sentences[:3])
        
        if len(response) > 1000:
            response = response[:1000]
            last_period = max(response.rfind('.'), response.rfind('!'), response.rfind('?'))
            if last_period > 0:
                response = response[:last_period + 1]
        
        return response
    
    def _insert_emotional_phrases(self, response: str, emotions: List[str]) -> str:
        need_template = len(response) < 20 or not any(phrase in response.lower() for phrase in PROMPT_CONFIG["empathy_phrases"])
        
        if need_template and emotions:
            primary_emotion = emotions[0]
            if primary_emotion in EMOTION_TEMPLATES:
                template = random.choice(EMOTION_TEMPLATES[primary_emotion])
                
                if response.startswith("Assistant:"):
                    prefix = response[:10]
                    rest = response[10:]
                    return f"{prefix} {template} {rest.lstrip()}"
                else:
                    return f"{template} {response}"
        
        return response
    
    def _apply_safety_filters(self, response: str) -> str:
        prohibited_patterns = [
            r"(?i)seek\s+professional\s+help",
            r"(?i)therapist",
            r"(?i)doctor",
            r"(?i)hotline",
            r"(?i)emergency\s+services",
            r"(?i)call\s+\d{3}",
            r"(?i)suicide\s+hotline",
            r"(?i)crisis\s+line",
            r"(?i)mental\s+health\s+professional"
        ]
        
        filtered_response = response
        for pattern in prohibited_patterns:
            filtered_response = re.sub(pattern, "support", filtered_response)
            
        return filtered_response
    
    def _update_conversation_state(self, user_msg: str, bot_response: str):
        """Update conversation state"""
        # Trust building (gradual increase with small decay)
        self.conversation_state["trust_level"] = min(
            0.95,
            self.conversation_state["trust_level"] * 0.95 + 0.05
        )
        
        # Intensity decay
        self.conversation_state["current_intensity"] *= 0.85
        
        # Adjust formality based on conversation progress
        if self.conversation_state["session_turns"] > 5:
            self.conversation_state["formality"] = max(0.1, self.conversation_state["formality"] * 0.9)

    def chat(self, user_message: str, user_id: str = "default_user") -> str:
        """Main chat interface with the enhanced RAG pipeline"""
        try:
            # 1. Classify query type first
            query_type = self.classify_query_type(user_message)
            print(f"Query type: {query_type}")
            
            # 2. Only detect emotions for emotional queries or chitchat
            emotions = []
            if query_type in [QUERY_TYPES["EMOTIONAL"], QUERY_TYPES["CHITCHAT"]]:
                emotions = self.detect_emotions(user_message)
                print(f"Detected emotions: {', '.join(emotions)}")
            else:
                emotions = ["neutral"]
            
            # 3. Generate appropriate response based on query type
            response = self.generate_response(user_message, query_type, emotions, user_id)
            
            # 4. Save conversation to database
            self.save_to_database(user_id, user_message, query_type, emotions, response)
            
            return response
            
        except Exception as e:
            print(f"Error in chat method: {e}")
            return "I'm having trouble processing that right now. Could you try saying that differently?"

In [ ]:
# =====================
# Execution
# =====================
if __name__ == "__main__":
    chatbot = EmotionChatbot()
    
    # Example interaction
    user_id = "test_user_1"
    print("Chatbot: Hello! How can I help you today?")
    
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("Chatbot: Goodbye! Take care.")
            break
            
        response = chatbot.chat(user_input, user_id)
        print(f"Chatbot: {response}")

Loading models...


Device set to use cuda:0
/home/jupyter-st125338/.local/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
Device set to use cuda:0


Loading Mistral 7B model...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/tmp/ipykernel_2516003/3972076662.py:116: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(


...

Query type: greeting


/home/jupyter-st125338/.local/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:679: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Chatbot: Hello there! I'm here to chat and offer a listening ear whenever you need. It sounds like you've been having some challenging thoughts recently. I encourage you to reach out to people you trust and talk about how you' re feeling. Remember that seeking help is a brave and important step towards taking care of both your emotional and physical wellbeing. Take things one day at a time, HermIT, and remember that it gets better. Is there anything else you'd feel comfortable discussing


You:  I am having strange feelings lately, like sadness and loneliness 


Error classifying query: TextClassificationPipeline.__call__() takes 2 positional arguments but 3 were given
Query type: chitchat
Detected emotions: sadness, nervousness, disappointment
Chatbot: I've noticed that you' ve mentioned feeling sadnessand lonelfulnessately. It's completelynormaltoexperiencetheseemotionsfromtime to time. However, if they're persisting for an extended period, it might be helpful to reachout to someone youtrust or a supportfor additional support. Inthe meantime,tryengaginginactivitiesthatbringyoujoyandconnectingwithlovedonesorcommunity.Remember,it'sokaytoaskforhelpwhenyouneedit,Hermitishereforyou.Isthereanythingelseyou'dliketalkabout?


In [ ]:
# # =====================
# # Execution Example
# # =====================
# if __name__ == "__main__":
#     chatbot = EmotionChatbot()
    
#     print("Emotion-Aware Chatbot initialized. Type 'exit' to quit.")
#     print("="*50)
    
#     user_id = input("Enter your user ID (or press Enter for default): ") or "default_user"
    
#     while True:
#         user_input = input("\nYou: ")
#         if user_input.lower() in ["exit", "quit", "bye"]:
#             print("Chatbot: Goodbye! Take care.")
#             break
            
#         response = chatbot.chat(user_input, user_id)
#         print(f"Chatbot: {response}")